In [1]:
import pandas as pd
import json

# List of all columns
l = [list(map(str.lower, pd.DataFrame.from_csv("data/csv/" + str(year) + ".csv").columns)) 
     for year in range(2004, 2013)]
s = set(l[0]).intersection(*l)

C:\Users\Rens\Anaconda\lib\site-packages\pandas\io\parsers.py:1139: DtypeWarning: Columns (75) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
C:\Users\Rens\Anaconda\lib\site-packages\pandas\io\parsers.py:1139: DtypeWarning: Columns (21,89,112,120) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
C:\Users\Rens\Anaconda\lib\site-packages\pandas\io\parsers.py:1139: DtypeWarning: Columns (79,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
C:\Users\Rens\Anaconda\lib\site-packages\pandas\io\parsers.py:1139: DtypeWarning: Columns (78,79,80,122,130) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [2]:
def get_json(year, item, areades = "RECS", index = None, location = "data/csv/"):
    """
    Returns json. All parameters are strings.
    
    Year: the year to retrieve json for.
    Item: the item (column name) to retrieve json for.
    Areades: the name of the column designating what type of area a row belongs to (wijk/buurt/gemeente).
    Index: the name of the column designating the !UNIQUE! name of a wijk/buurt/gemeente.
    
    E.g.
    
    get_json("2003", "AANT_MAN", "RECS", "GWB_NAAM03_40pos")
    """
    # If no index is passed, assuming year range of 2004-2012.
    if not index:
        index = "GWB_NAAM" + year[2:] + "_40pos"
    
    data = pd.DataFrame.from_csv(location + year + ".csv")
    
    # Lower all input and all columns (CBS uses lower/upper case for its columns throughout the years).
    item, areades, index = item.lower(), areades.lower(), index.lower()
    data.columns = map(str.lower, data.columns)
    
    # First select only all gemeentes, then filter based on given item and index. Set given index.
    df = data[(data[areades] == 'Gemeente') | (data[areades] == "G")].filter([item, index]).set_index(index)
    
    # Rename item column to year colum.
    df = df.rename(columns = {item: year})
    return df.to_json()

In [3]:
for item in s:
    item = str(item)
    data = {}
    for year in range(2004, 2013):
        print item, year
        year = str(year)
        data.update(json.loads(get_json(year, item, "RECS", index = None)))
        
    with open('data/json/' + item + '.json', 'w') as f:
        json.dump({item + ".json": data}, f)

opp_tot 2004
opp_tot 2005
opp_tot 2006
opp_tot 2007
opp_tot 2008
opp_tot 2009
opp_tot 2010
opp_tot 2011
opp_tot 2012
bedr_auto 2004
bedr_auto 2005
bedr_auto 2006
bedr_auto 2007
bedr_auto 2008
bedr_auto 2009
bedr_auto 2010
bedr_auto 2011
bedr_auto 2012
dek_perc 2004
dek_perc 2005
dek_perc 2006
dek_perc 2007
dek_perc 2008
dek_perc 2009
dek_perc 2010
dek_perc 2011
dek_perc 2012
p_over_nw 2004
p_over_nw 2005
p_over_nw 2006
p_over_nw 2007
p_over_nw 2008
p_over_nw 2009
p_over_nw 2010
p_over_nw 2011
p_over_nw 2012
aant_inw 2004
aant_inw 2005


KeyboardInterrupt: 

In [ ]:
"""
Index:
----------------------
2003: GWB_NAAM03_40pos
2004: GWB_NAAM04_40pos
2005: GWB_NAAM05_40pos
2006: GWB_NAAM06_40pos
2007: GWB_NAAM07_40pos
2008: GWB_NAAM08_40pos
2009: GWB_NAAM09_40pos
2010: GWB_NAAM10_40pos
2011: GWB_NAAM11_40pos
2012: GWB_NAAM12_40POS
2013: regio
2014: regio


Areades:
----------------------
2003: RECS
2004: RECS
2005: RECS
2006: RECS
2007: RECS
2008: RECS
2009: RECS
2010: RECS
2011: RECS
2012: RECS (== G)
2013: recs (== Gemeente)
2014: recs (== Gemeente)
"""
print